In [2]:
!pip install yolox --no-deps  # onnxruntime==1.8.0 is putdated, hence --no-deps
!git clone https://github.com/mikel-brostrom/boxmot.git
!cd boxmot
!pip install poetry
!poetry install --with yolo  # installed boxmot + yolo dependencies
!poetry shell  # activates the newly created environment with the installed dependencies



Cloning into 'boxmot'...
remote: Enumerating objects: 15716, done.
remote: Counting objects: 100% (334/334), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 15716 (delta 206), reused 277 (delta 192), pack-reused 15382 (from 1)
Receiving objects: 100% (15716/15716), 118.02 MiB | 12.13 MiB/s, done.
Resolving deltas: 100% (10110/10110), done.
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl (54 kB)
Using cached shellingham-1.5.4-py2.py3-none-any.whl (9.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 7.4 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 3.1 MB/s eta 0:00:00-:--:--

Poetry could not find a pyproject.toml file in /Users/marcuscanhaco/workspace/VisionStudy_TC4/notebooks or its parents

Poetry could not find a pyproject.toml file in /Users/marcuscanhaco/w

In [4]:
!pip install tabulate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yolox 0.3.0 requires ninja, which is not installed.
yolox 0.3.0 requires onnx==1.8.1, which is not installed.
yolox 0.3.0 requires onnx-simplifier==0.3.5, which is not installed.
yolox 0.3.0 requires onnxruntime==1.8.0, which is not installed.
yolox 0.3.0 requires pycocotools>=2.0.2, which is not installed.
yolox 0.3.0 requires scikit-image, which is not installed.
yolox 0.3.0 requires thop, which is not installed.


In [6]:
import cv2
import gdown
import torch
import numpy as np
from pathlib import Path
from torchvision import transforms
from ultralytics.utils import ops

from yolox.exp import get_exp
from yolox.utils import postprocess
from yolox.utils.model_utils import fuse_model
from boxmot import BotSort
from boxmot.utils.ops import yolox_preprocess


# Dictionary for YOLOX model weights URLs
YOLOX_ZOO = {
    'yolox_n.pt': 'https://drive.google.com/uc?id=1AoN2AxzVwOLM0gJ15bcwqZUpFjlDV1dX',
    'yolox_s.pt': 'https://drive.google.com/uc?id=1uSmhXzyV1Zvb4TJJCzpsZOIcw7CCJLxj',
    'yolox_m.pt': 'https://drive.google.com/uc?id=11Zb0NN_Uu7JwUd9e6Nk8o2_EUfxWqsun',
    'yolox_l.pt': 'https://drive.google.com/uc?id=1XwfUuCBF4IgWBWK2H7oOhQgEj9Mrb3rz',
    'yolox_x.pt': 'https://drive.google.com/uc?id=1P4mY0Yyd3PPTybgZkjMYhFri88nTmJX5',
}

# Preprocessing pipeline
input_size = [800, 1440]
device = torch.device('cpu')
yolox_model = 'yolox_s.pt'
yolox_model_path = Path(yolox_model)

# Download model if not present
if not yolox_model_path.exists():
    gdown.download(YOLOX_ZOO[yolox_model], output=str(yolox_model_path), quiet=False)

# Initialize YOLOX model
exp = get_exp(None, 'yolox_s')
exp.num_classes = 1
ckpt = torch.load(yolox_model_path, map_location=device)

model = exp.get_model()
model.load_state_dict(ckpt["model"])
model = fuse_model(model).to(device).eval()

# Initialize tracker
tracker = BotSort(reid_weights=Path('osnet_x0_25_msmt17.pt'), device=device, half=False)

2024-11-26 11:30:07.015 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v11.0.5 🚀 Python-3.12.4 torch-2.2.2CPU
Downloading...
From: https://drive.google.com/uc?id=1sSwXSUlj4_tHZequ_iZ8w_Jh0VaRQMqF
To: /Users/marcuscanhaco/workspace/VisionStudy_TC4/notebooks/osnet_x0_25_msmt17.pt

  0%|          | 0.00/3.06M [00:00<?, ?B/s]
 17%|█▋        | 524k/3.06M [00:00<00:00, 3.03MB/s]
100%|██████████| 3.06M/3.06M [00:00<00:00, 10.1MB/s]
2024-11-26 11:30:11.884 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:183 - Loaded pretrained weights from osnet_x0_25_msmt17.pt


In [9]:
from yolox.utils import vis
import os

# Diretórios
input_dir = "/Users/marcuscanhaco/workspace/VisionStudy_TC4/data/Frames_extracted"
output_dir = "/Users/marcuscanhaco/workspace/VisionStudy_TC4/data/Frames_annotated"

# Criar o diretório de saída, se não existir
os.makedirs(output_dir, exist_ok=True)
frame = None
# Processar cada frame PNG na pasta
for img_name in os.listdir(input_dir):
    if img_name.endswith(".png"):
        # Carregar a imagem
        img_path = os.path.join(input_dir, img_name)
        frame = cv2.imread(img_path)
        if frame is None:
            print(f"Erro ao carregar {img_name}")
            continue

    # Preprocess frame
    height, width = frame.shape[:2]
    img_info = {"height": height, "width": width, "raw_img": frame}
    frame_img, ratio = yolox_preprocess(frame, input_size=input_size)
    frame_tensor = torch.Tensor(frame_img).unsqueeze(0).to(device)

    # Detection with YOLOX
    with torch.no_grad():
        output = model(frame_tensor)
        output = postprocess(output, 1, 0.5, 0.7, class_agnostic=True)[0]


    # Visualização e salvamento
    if output[0] is not None:
        bboxes = output[0][:, 0:4]
        cls_ids = output[0][:, 6]
        scores = output[0][:, 4] * output[0][:, 5]
    
        # Desenhar caixas delimitadoras e IDs na imagem
        annotated_img = vis(img_info["raw_img"], bboxes, scores, cls_ids, conf=exp.test_conf)
    
        # Salvar imagem anotada
        output_path = os.path.join(output_dir, img_name)
        cv2.imwrite(output_path, annotated_img)
        print(f"Imagem anotada salva em: {output_path}")
    else:
        print(f"Nenhuma detecção em {img_name}")

TypeError: 'NoneType' object is not subscriptable